# Prededecir recursivo con rolling means por categoria

## Data

In [1]:
from DataLoader import importar_ventas,leer_csv
ventas=importar_ventas()


In [2]:
from DataLoader import importar_ventas,leer_csv
len(leer_csv("eci_product_master")["subgroup"].unique())

74

In [3]:
TARGET = ["total_sales"]
FEATURES_PRIMITIVAS = ['date','day', 'month',
                       "store_cod",'subgroup_cod', 'year',"cluster","mean_price" ]

FEATURES=[
    'subgroup_cod', 'total_sales_lag_1', 'year',
    'anual_sin_1', 'anual_cos_1', 'mensual_sin_1', 'mensual_sin_2',
    'mean_price_rolling_mean_7d', 'mean_price_rolling_sum_14d',
    'total_sales_rolling_mean_7d', 'total_sales_rolling_sum_7d',
    'total_sales_rolling_mean_14d', 'total_sales_rolling_sum_14d'
]

In [4]:
def obtener_buffers(ventas):
    subgroups=ventas["subgroup_cod"].unique()
    stores=ventas["store_cod"].unique()
    buffers={}
    ventas=ventas.sort_values(["subgroup_cod","store_cod","date"])
    for subgroup in subgroups:
        for store in stores:
            data=ventas[((ventas["subgroup_cod"]==subgroup) & (ventas["store_cod"]==store))]
            buffers[(subgroup,store,"mean_price")]= data["mean_price"].tail(14).to_numpy()
            buffers[(subgroup,store,"total_sales")]=data["total_sales"].tail(14).to_numpy()
    return buffers

In [5]:
buffers=obtener_buffers(ventas)


In [6]:
from DataLoader import generar_subgroup_cod,generar_store_cod, leer_csv,subgroup_category
from DataLoader import agregar_fourier
import pandas as pd
rolling_features=['mean_price_rolling_mean_7d', 'mean_price_rolling_sum_14d',
    'total_sales_rolling_mean_7d', 'total_sales_rolling_sum_7d',
    'total_sales_rolling_mean_14d', 'total_sales_rolling_sum_14d',"total_sales_lag_1"]


def test_recursivo(rolling_features,buffers):
    test=leer_csv("ids_test.csv")
    test[['store_id', 'subgroup', 'date']] = test["store_subgroup_date_id"].str.split('_', expand=True)
    test["store_cod"]=test["store_id"].map(generar_store_cod())
    test["subgroup_cod"]=test["subgroup"].map(generar_subgroup_cod())
    test["date"]=pd.to_datetime(test["date"])
    test.index=test["date"]
    dia_inicial= pd.to_datetime("2024-01-01")
    iniciar=test.loc[test["date"] == dia_inicial, ["subgroup_cod","store_cod"]]
    
    for feature in rolling_features:
        test[feature]=0.0 
    
    
    for index,row in iniciar.iterrows():
        if row["subgroup_cod"]!=32: #basketball
            buffer_total_sales=buffers[(row["subgroup_cod"],row["store_cod"],"total_sales")]
            buffer_mean_price=buffers[(row["subgroup_cod"],row["store_cod"],"mean_price")]
            for feature in rolling_features:
                if "total_sales" in feature:
                    if "rolling_mean" in feature:
                        if "7" in feature:
                            test.loc[index,feature]=buffer_total_sales[len(buffer_total_sales)-7:len(buffer_total_sales)].mean()
                        else:
                            test.loc[index,feature]=buffer_total_sales[len(buffer_total_sales)-14:len(buffer_total_sales)].mean()
                    
                    elif "rolling_sum" in feature:
                        if "7" in feature:                   
                            test.loc[index,feature]=buffer_total_sales[len(buffer_total_sales)-7:len(buffer_total_sales)].sum()
                        else:
                            test.loc[index,feature]=buffer_total_sales[len(buffer_total_sales)-14:len(buffer_total_sales)].sum()
                    elif "lag_1" in feature:
                        test.loc[index,feature]=buffer_total_sales[-1]
                else:
                    if "rolling_mean" in feature:
                        if "7" in feature:
                            test.loc[index,feature]=buffer_mean_price[len(buffer_mean_price)-7:len(buffer_mean_price)].mean()
                        else:  
                            test.loc[index,feature]=buffer_mean_price[len(buffer_mean_price)-14:len(buffer_mean_price)].mean()
                    else:
                        if "7" in feature:                                           
                            test.loc[index,feature]=buffer_mean_price[len(buffer_mean_price)-7:len(buffer_mean_price)].sum()
                        else:
                            test.loc[index,feature]=buffer_mean_price[len(buffer_mean_price)-14:len(buffer_mean_price)].sum()

        
        test["category"]=test["subgroup"].map(subgroup_category())
        return test
test=test_recursivo(['mean_price_rolling_mean_7d', 'mean_price_rolling_sum_14d',
    'total_sales_rolling_mean_7d', 'total_sales_rolling_sum_7d',
    'total_sales_rolling_mean_14d', 'total_sales_rolling_sum_14d',"total_sales_lag_1"],buffers)
test




agregar_fourier(test,2)

In [7]:
from DataLoader import agregar_fourier,generar_rolling_features,generar_lag_features
agregar_fourier(ventas,2)
ventas=generar_rolling_features(ventas,"mean_price",[1,2],["subgroup_cod","store_cod"])
ventas=generar_rolling_features(ventas,"total_sales",[1,2],["subgroup_cod","store_cod"])
generar_lag_features(ventas,"total_sales",1)
test["year"]=test["date"].dt.year

In [8]:
test=test.reset_index(drop=True)
test

,store_subgroup_date_id,store_id,subgroup,date,store_cod,subgroup_cod,mean_price_rolling_mean_7d,mean_price_rolling_sum_14d,total_sales_rolling_mean_7d,total_sales_rolling_sum_7d,...,category,anual_sin_1,anual_cos_1,mensual_sin_1,mensual_cos_1,anual_sin_2,anual_cos_2,mensual_sin_2,mensual_cos_2,year
0,S00001_Laptops_2024-01-01,S00001,Laptops,2024-01-01,0,0,289.963492,4081.195778,1307.287143,9151.01,...,Electronics,0.017202,0.999852,0.20495,0.978773,0.034398,0.999408,0.401198,0.915991,2024
1,S00001_Laptops_2024-01-02,S00001,Laptops,2024-01-02,0,0,0.000000,0.000000,0.000000,0.00,...,Electronics,0.034398,0.999408,0.20495,0.978773,0.068755,0.997634,0.401198,0.915991,2024
2,S00001_Laptops_2024-01-03,S00001,Laptops,2024-01-03,0,0,0.000000,0.000000,0.000000,0.00,...,Electronics,0.051584,0.998669,0.20495,0.978773,0.103031,0.994678,0.401198,0.915991,2024
3,S00001_Laptops_2024-01-04,S00001,Laptops,2024-01-04,0,0,0.000000,0.000000,0.000000,0.00,...,Electronics,0.068755,0.997634,0.20495,0.978773,0.137185,0.990545,0.401198,0.915991,2024
4,S00001_Laptops_2024-01-05,S00001,Laptops,2024-01-05,0,0,0.000000,0.000000,0.000000,0.00,...,Electronics,0.085906,0.996303,0.20495,0.978773,0.171177,0.985240,0.401198,0.915991,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77695,S00157_Puzzles_2024-01-03,S00157,Puzzles,2024-01-03,156,73,0.000000,0.000000,0.000000,0.00,...,Toys,0.051584,0.998669,0.20495,0.978773,0.103031,0.994678,0.401198,0.915991,2024
77696,S00157_Puzzles_2024-01-04,S00157,Puzzles,2024-01-04,156,73,0.000000,0.000000,0.000000,0.00,...,Toys,0.068755,0.997634,0.20495,0.978773,0.137185,0.990545,0.401198,0.915991,2024
77697,S00157_Puzzles_2024-01-05,S00157,Puzzles,2024-01-05,156,73,0.000000,0.000000,0.000000,0.00,...,Toys,0.085906,0.996303,0.20495,0.978773,0.171177,0.985240,0.401198,0.915991,2024
77698,S00157_Puzzles_2024-01-06,S00157,Puzzles,2024-01-06,156,73,0.000000,0.000000,0.000000,0.00,...,Toys,0.103031,0.994678,0.20495,0.978773,0.204966,0.978769,0.401198,0.915991,2024


In [ ]:
import xgboost as xgb
import datetime as dt
import numpy as np
model = xgb.XGBRegressor( 
    n_estimators=500,           # alto para usar early stopping
    learning_rate=0.05,          # más bajo para no memorizar rápido
    max_depth=4,                 # menos profundidad => menos overfit
    #min_child_weight=5,          # evita que splits se hagan con poca data
    #subsample=0.8,                # usa solo parte de las filas en cada árbol
    colsample_bytree=1,         # usa solo parte de las columnas en cada árbol
    reg_lambda=1,                 # L2 regularization
    reg_alpha=20,                  # L1 regularization
   # gamma=0.2,                   # penaliza splits innecesarios
    random_state=42,
    n_jobs=-1,
                        )
TARGET = ["total_sales"]
FEATURES=[
    'subgroup_cod', 'total_sales_lag_1', 'year',
    'anual_sin_1', 'anual_cos_1', 'mensual_sin_1', 'mensual_sin_2',
    'mean_price_rolling_mean_7d', 'mean_price_rolling_sum_14d',
    'total_sales_rolling_mean_7d', 'total_sales_rolling_sum_7d',
    'total_sales_rolling_mean_14d', 'total_sales_rolling_sum_14d'
]
ventas["total_sales"]=ventas["total_sales"].fillna(0)
train_cat=ventas[["category"]+FEATURES]
separador = "category"
bins = ventas[separador].unique()
test["pred"]=0.0
for bucket in bins:
    data_bucket=ventas[ventas[separador]==bucket]
    dias_restantes=test
    X_train=data_bucket[FEATURES]
    y_train=data_bucket[TARGET]
    model.fit(X_train,y_train)
    for i in range(7):
        siguiente=dias_restantes.groupby(["subgroup_cod","store_cod"]).head(1)
        dias_restantes=dias_restantes.drop(siguiente.index)
        print(f"dias predichos: {i}")
        prediccion=model.predict(siguiente[FEATURES])
        siguiente.loc[:,"pred"]=prediccion
        lag=siguiente["pred"]
        test.loc[siguiente.index,"pred"]=siguiente["pred"]
        if i<6:
            lag.index=siguiente.index+1
            dias_restantes.loc[lag.index,"total_sales_lag_1"]=lag
            
            for index,row in dias_restantes.head(1).iterrows():
                if row["subgroup_cod"]!=32: #basketball
                    total_sales_pred=siguiente[((siguiente["subgroup_cod"]==row["subgroup_cod"])& (siguiente["store_cod"]==row["store_cod"]))]["pred"].to_numpy()
                    buffers[(row["subgroup_cod"],row["store_cod"],"total_sales")]=np.concatenate([buffers[(row["subgroup_cod"],row["store_cod"],"total_sales")],total_sales_pred])
                    buffer_total_sales=buffers[(row["subgroup_cod"],row["store_cod"],"total_sales")]
                    
                    buffers[(row["subgroup_cod"],row["store_cod"],"mean_price")]=np.concatenate([buffers[(row["subgroup_cod"],row["store_cod"],"mean_price")],[buffers[(row["subgroup_cod"],row["store_cod"],"mean_price")][-1]*(1+np.random.uniform(-0.1, 0.1))]])
                    buffer_mean_price=buffers[(row["subgroup_cod"],row["store_cod"],"mean_price")]
                                                                
                    for feature in rolling_features:
                        if "total_sales" in feature:
                            if "rolling_mean" in feature:
                                if "7" in feature:
                                    dias_restantes.loc[index,feature]=buffer_total_sales[len(buffer_total_sales)-7:len(buffer_total_sales)].mean()
                                else:
                                    dias_restantes.loc[index,feature]=buffer_total_sales[len(buffer_total_sales)-14:len(buffer_total_sales)].mean()
                            
                            elif "rolling_sum" in feature:
                                if "7" in feature:                   
                                    dias_restantes.loc[index,feature]=buffer_total_sales[len(buffer_total_sales)-7:len(buffer_total_sales)].sum()
                                else:
                                    dias_restantes.loc[index,feature]=buffer_total_sales[len(buffer_total_sales)-14:len(buffer_total_sales)].sum()
                            elif "lag_1" in feature:
                                dias_restantes.loc[index,feature]=buffer_total_sales[-1]
                        else:
                            if "rolling_mean" in feature:
                                if "7" in feature:
                                    dias_restantes.loc[index,feature]=buffer_mean_price[len(buffer_mean_price)-7:len(buffer_mean_price)].mean()
                                else:  
                                    dias_restantes.loc[index,feature]=buffer_mean_price[len(buffer_mean_price)-14:len(buffer_mean_price)].mean()
                            else:
                                if "7" in feature:                                           
                                    dias_restantes.loc[index,feature]=buffer_mean_price[len(buffer_mean_price)-7:len(buffer_mean_price)].sum()
                                else:
                                    dias_restantes.loc[index,feature]=buffer_mean_price[len(buffer_mean_price)-14:len(buffer_mean_price)].sum()
        


In [ ]:
test[test["pred"].isna()]

In [ ]:
resultados=dict(zip(test["store_subgroup_date_id"],test["pred"]))
kaggle=pd.read_csv("ids_test.csv")
kaggle["TOTAL_SALES"]=kaggle["STORE_SUBGROUP_DATE_ID"].map(resultados)

In [ ]:
kaggle.to_csv("Pred_recursiva_xgboost.csv",index=False)